In [12]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [3]:
pip install tensorflow_datasets

  Obtaining dependency information for tensorflow_datasets from https://files.pythonhosted.org/packages/a1/73/7a9ed7935f6833d73b32f1e2a1210082f5ccb95445440b4e2b0f66ab7792/tensorflow_datasets-4.9.3-py3-none-any.whl.metadata
  Obtaining dependency information for array-record from https://files.pythonhosted.org/packages/93/8e/c6b646029a9c544ecd6806c6ea0efb67d0ba3300e1d68518f58d9774d3f0/array_record-0.4.1-py310-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.4/115.4 kB 1.7 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for etils[enp,epath,etree]>=0.9.0 from https://files.pythonhosted.org/packages/0f/6a/d2aaebacf73d5da7126c632ec0d9dc2df99cc4bbd259bad48904a034fc1b/etils-1.5.2-py3-none-any.whl.metadata
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for tensorflow-metadata from https://files.pythonhosted.org/packages/41/23/3705c7139886c079ef4c0e3be56a5a1fb90e9ee413a4b7caaee0ee0ea6fe/tensorflow_metadata-1.14.0-py3-none-any.whl.m

In [4]:
import tensorflow_datasets as tfds

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
(train, test) = tfds.load('mnist',
                                split = ['train','test'])

In [8]:
train

<_PrefetchDataset element_spec={'image': TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None)}>

In [9]:
test

<_PrefetchDataset element_spec={'image': TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None)}>

In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

def load_dataset(train_size, test_size):
    train, test = tf.keras.datasets.mnist.load_data()

    # take a sample
    train_idx = np.random.randint(low=0, high=train[0].shape[0], size=train_size)
    test_idx = np.random.randint(low=0, high=test[0].shape[0], size=test_size)
    X_train = train[0][train_idx].reshape(-1,28*28)
    y_train = train[1][train_idx].reshape(-1,1)
    X_test = test[0][test_idx].reshape(-1,28*28)
    y_test = test[1][test_idx].reshape(-1,1)

    scale = StandardScaler()
    X_train = scale.fit_transform(X_train)
    X_test = scale.transform(X_test)

    OH = OneHotEncoder(categories='auto', sparse=False)
    y_train = OH.fit_transform(y_train)
    y_test = OH.transform(y_test)

    print('X_train:',X_train.shape)
    print('y_train:',y_train.shape)
    print('X_test:',X_test.shape)
    print('y_test:',y_test.shape)
    print('Min:', X_train.min())
    print('Max:', X_train.max())

    return X_train, X_test, y_train, y_test


In [14]:
X_train, X_test, y_train, y_test = load_dataset(60000,10000)

X_train: (60000, 784)
y_train: (60000, 10)
X_test: (10000, 784)
y_test: (10000, 10)
Min: -1.2716186854969616
Max: 244.94693302857712


/usr/local/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


### Optimizing at the Initialization Stage

In [20]:
from tensorflow.keras import layers

In [21]:
def simple_model(initializer, activation='relu'):
    return tf.keras.Sequential([layers.Dense(32, activation, input_shape=(784,), kernel_initializer=initializer),
                                layers.Dense(32, activation, kernel_initializer=initializer),
                                layers.Dense(32, activation, kernel_initializer=initializer),
                                layers.Dense(32, activation, kernel_initializer=initializer),
                                layers.Dense(10, activation = 'softmax', kernel_initializer=tf.keras.initializers.glorot_normal())



    ])

In [22]:
init = tf.initializers.zeros()
activate = 'relu'
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
model_zeros = simple_model(init, activate)
model_zeros.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model_zeros.fit(X_train, y_train, epochs=10, batch_size=3200, validation_data=(X_test, y_test))

Epoch 1/10
19/19 [==============================] - 1s 36ms/step - loss: 2.3025 - accuracy: 0.1155 - val_loss: 2.3025 - val_accuracy: 0.1144
Epoch 2/10
19/19 [==============================] - 0s 15ms/step - loss: 2.3025 - accuracy: 0.1163 - val_loss: 2.3025 - val_accuracy: 0.1144
Epoch 3/10
19/19 [==============================] - 0s 13ms/step - loss: 2.3024 - accuracy: 0.1163 - val_loss: 2.3024 - val_accuracy: 0.1144
Epoch 4/10
19/19 [==============================] - 0s 13ms/step - loss: 2.3023 - accuracy: 0.1163 - val_loss: 2.3024 - val_accuracy: 0.1144
Epoch 5/10
19/19 [==============================] - 0s 15ms/step - loss: 2.3022 - accuracy: 0.1163 - val_loss: 2.3023 - val_accuracy: 0.1144
Epoch 6/10
19/19 [==============================] - 0s 15ms/step - loss: 2.3022 - accuracy: 0.1163 - val_loss: 2.3023 - val_accuracy: 0.1144
Epoch 7/10
19/19 [==============================] - 0s 12ms/step - loss: 2.3021 - accuracy: 0.1163 - val_loss: 2.3022 - val_accuracy: 0.1144
Epoch 8/10
19

In [25]:
init = tf.initializers.random_normal()
activate = 'relu'
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
model_normal = simple_model(init, activate)
model_normal.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model_normal.fit(X_train, y_train, epochs=10, batch_size=3200, validation_data=(X_test, y_test))

/usr/local/lib/python3.11/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Epoch 1/10
19/19 [==============================] - 1s 37ms/step - loss: 2.3027 - accuracy: 0.0958 - val_loss: 2.3024 - val_accuracy: 0.1091
Epoch 2/10
19/19 [==============================] - 0s 16ms/step - loss: 2.3022 - accuracy: 0.1204 - val_loss: 2.3021 - val_accuracy: 0.1174
Epoch 3/10
19/19 [==============================] - 0s 14ms/step - loss: 2.3020 - accuracy: 0.1180 - val_loss: 2.3019 - val_accuracy: 0.1154
Epoch 4/10
19/19 [==============================] - 0s 14ms/step - loss: 2.3017 - accuracy: 0.1167 - val_loss: 2.3017 - val_accuracy: 0.1150
Epoch 5/10
19/19 [==============================] - 0s 13ms/step - loss: 2.3015 - accuracy: 0.1164 - val_loss: 2.3015 - val_accuracy: 0.1147
Epoch 6/10
19/19 [==============================] - 0s 12ms/step - loss: 2.3013 - accuracy: 0.1163 - val_loss: 2.3014 - val_accuracy: 0.1144
Epoch 7/10
19/19 [==============================] - 0s 11ms/step - loss: 2.3012 - accuracy: 0.1163 - val_loss: 2.3012 - val_accuracy: 0.1144
Epoch 8/10
19

In [26]:
init = tf.initializers.glorot_uniform()
activate = 'relu'
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
model_glorotu = simple_model(init, activate)
model_glorotu.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model_glorotu.fit(X_train, y_train, epochs=10, batch_size=3200, validation_data=(X_test, y_test))

/usr/local/lib/python3.11/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Epoch 1/10
19/19 [==============================] - 1s 27ms/step - loss: 2.3383 - accuracy: 0.1120 - val_loss: 2.3176 - val_accuracy: 0.1361
Epoch 2/10
19/19 [==============================] - 0s 12ms/step - loss: 2.2861 - accuracy: 0.1450 - val_loss: 2.2699 - val_accuracy: 0.1649
Epoch 3/10
19/19 [==============================] - 0s 12ms/step - loss: 2.2421 - accuracy: 0.1782 - val_loss: 2.2267 - val_accuracy: 0.1920
Epoch 4/10
19/19 [==============================] - 0s 14ms/step - loss: 2.2000 - accuracy: 0.2051 - val_loss: 2.1837 - val_accuracy: 0.2134
Epoch 5/10
19/19 [==============================] - 0s 12ms/step - loss: 2.1565 - accuracy: 0.2286 - val_loss: 2.1393 - val_accuracy: 0.2330
Epoch 6/10
19/19 [==============================] - 0s 13ms/step - loss: 2.1104 - accuracy: 0.2500 - val_loss: 2.0923 - val_accuracy: 0.2520
Epoch 7/10
19/19 [==============================] - 0s 15ms/step - loss: 2.0608 - accuracy: 0.2695 - val_loss: 2.0419 - val_accuracy: 0.2709
Epoch 8/10
19